Here we have first the latest and last solution we applied for classification. <br>
After preparation, we made Binary and Multi-class classifiers with different algorithms:  DecisionTreeClassifier, LogisticRegression, RandomForestClassifier, GradientBoostingClassifier, XGBClassifier, MLPClassifier <br>
Also we compared each of them in 4 versions: <br>
1 - without appling any weighting or oversamplling <br>
2 - with applying custom weights <br>
3 - with applying class weights <br>
4 - with oversamplelling <br>

Finally we chose, XGBClassifier with class weight for our Binary classifier, final Logistic Regression with oversampling for our final Multi-class classifier <br>

At the end one more time we showed our inital solution which was flat classification with the following labels: 
labels' value count: label <br>
5    19250 <br>
0      878 <br>
1      721 <br>
2      718 <br>
4      662 <br>
3      447 <br>
In this solution we could not apply efficiantly apply the methods for solving imbalance issues.<br>
That led us to have instead two different classifiers (binary, multi-class)


In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
random_seed = 42
random.seed(random_seed)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import export_graphviz
import graphviz
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer, balanced_accuracy_score, f1_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from imblearn.over_sampling import RandomOverSampler

# Binary and Multi-class classification

In [4]:
# read data *****
final_episodes = pd.read_csv("/home/kabank/workbench/.conda/analysis/kabank-data/final_episodes.csv")
fullHot_episodes = pd.read_csv("/home/kabank/workbench/.conda/analysis/kabank-data/final_episodes4.csv") 
Diag_med = pd.read_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/Dummies_ICD10_ATC_100.csv') 
# required changes *****
final_episodes.rename(columns={'var_no_dates_permonth': 'SD_CareEvent_PerMonth'}, inplace=True)

#merge to add some required columns *****
final_episodes = pd.merge(final_episodes, fullHot_episodes[['episode_id', 'num_diagnoses', 'num_medications', 'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']], on='episode_id', how='inner')
final_episodes = pd.merge(final_episodes[['episode_id','tillnextepisode','Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']], Diag_med, on='episode_id', how='left')

# features = ['episode_id','Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
#        'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
#        'gender_0','F','M','MiddleChildhood','Preschooler','Teenager','tillnextepisode']
features = final_episodes.iloc[:,:].columns
print(features)
#final_episodes1 = final_episodes[features]


#labeling 0 and 1 for not_readmitted and readmitted
for i in range(len(final_episodes['tillnextepisode'])):
    if np.isnan(final_episodes.loc[i, 'tillnextepisode']):
        final_episodes.loc[i, 'binary_label'] = 0
    else:
        final_episodes.loc[i, 'binary_label'] = 1
    
final_episodes['binary_label'] = final_episodes['binary_label'].astype(int)

final_episodes1_episode_id = pd.DataFrame()
final_episodes1_episode_id['episode_id'] = final_episodes['episode_id'].astype(int)
#final_episodes1 = final_episodes[final_episodes.iloc[1,:].columns]
final_episodes1 = final_episodes.iloc[:, 1:]

final_episodes2_episode_id = pd.DataFrame()
final_episodes2_episode_id['episode_id'] = final_episodes[final_episodes['binary_label']==1]['episode_id'].astype(int)
final_episodes2 = final_episodes[final_episodes['binary_label']==1].iloc[:, 1:]
print('here:',final_episodes['episode_id'].nunique())
print('here:',final_episodes2_episode_id['episode_id'].nunique())
# Encoding 'tillnextepisode' into labels:  
le = LabelEncoder()
#First attempt with 6 labels and flat classification
# final_episodes2['multi_label'] = le.fit_transform(pd.cut(final_episodes2['tillnextepisode'],
#                                                   bins=[float('-inf'), 0, 180, 365, 730, 1095, float('inf')],
#                                                   labels=['not-re-admitted',
#                                                           're-admitted in 0-180 days',
#                                                           're-admitted in 180-365 days',
#                                                           're-admitted in 365-730 days',
#                                                           're-admitted in 730-1095 days',
#                                                           're-admitted in more than 1095 days']))

# test these: 
# bins=[float('-inf'), 0, 182, 730, float('inf')]
# bins=[float('-inf'), 0, 273, 730, float('inf')]
# bins=[float('-inf'), 0, 365, 730, float('inf')]
final_episodes2['multi_label'] = le.fit_transform(pd.cut(final_episodes2['tillnextepisode'],
                                                  bins=[float('-inf'), 0, 182, 730, float('inf')],
                                                  labels=['not-re-admitted',
                                                          '0-6m',
                                                          '6m-2y',
                                                          'over 2y']))

# final_episodes2['multi_label'] = le.fit_transform(pd.cut(final_episodes2['tillnextepisode'],
#                                                   bins=[float('-inf'), 0, 273, 730, float('inf')],
#                                                   labels=['not-re-admitted',
#                                                           '0-9m',
#                                                           '9m-2y',
#                                                           'over 2y']))

# final_episodes2['multi_label'] = le.fit_transform(pd.cut(final_episodes2['tillnextepisode'],
#                                                   bins=[float('-inf'), 0, 365, 730, float('inf')],
#                                                   labels=['not-re-admitted',
#                                                           '0-1y',
#                                                           '1y-2y',
#                                                           'over 2y']))



# for binary classification: admitted: 1/ not-readmitted: 0
#print(final_episodes1.columns)
print(len(final_episodes1))
print(final_episodes1['binary_label'].value_counts())
print(final_episodes1['binary_label'].unique())
counts = final_episodes1['binary_label'].value_counts()
print(counts[0], counts[1])
max_val_bin = counts.max()
print(counts.idxmax(),'with', max_val_bin)

print(len(final_episodes2))
print(final_episodes2['multi_label'].value_counts())
print(final_episodes2['multi_label'].unique())
counts = final_episodes2['multi_label'].value_counts()
max_val_multi = counts.max()
print(max_val_multi)
print(counts.idxmax(),'with', max_val_multi)

print(len(final_episodes1), final_episodes1.columns)
print(len(final_episodes2), final_episodes2.columns)
print(len(final_episodes1_episode_id))
print(len(final_episodes2_episode_id))


Index(['episode_id', 'tillnextepisode', 'Length_of_Episode', 'Count_visit',
       'SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
       'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio',
       ...
       'G03DA02', 'B03BA03', 'G03FB05', 'S01AA13', 'A11JB', 'M01AE01',
       'N05AE04', 'N06AA09', 'R03AC03', 'B03BA01'],
      dtype='object', length=218)
here: 22676
here: 3426
22676
binary_label
0    19250
1     3426
Name: count, dtype: int64
[0 1]
19250 3426
0 with 19250
3426
multi_label
1    1316
2    1109
0    1001
Name: count, dtype: int64
[1 2 0]
1316
1 with 1316
22676 Index(['tillnextepisode', 'Length_of_Episode', 'Count_visit',
       'SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
       'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio',
       'TreatmentPlanning_ratio',
       ...
       'B03BA03', 'G03FB05', 'S01AA13', 'A11JB', 'M01AE01', 'N05AE04',
       'N06AA09', 'R03AC03', 'B03BA01', 'binary_label'],
      dtype='object', lengt

In [5]:
# Find the weights for Random Forest
# https://www.kaggle.com/code/christiantheilhave/class-imbalance-with-weighted-random-forest/notebook
final_episodes1.columns = [str(col) for col in final_episodes1.columns]

#features = final_episodes1.iloc[:, 1:-1].columns
#X = final_episodes1[features]

X = final_episodes1.iloc[:, 1:-1]
yb = final_episodes1['binary_label']

# Remove rows with NaN in the target variable
#X.columns = X.columns.astype(str)
X = X.copy()
X.fillna(0, inplace=True)
X = X[~yb.isna()]
yb = yb.dropna()

# Splitting the data into training and testing sets *****
X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                yb,
                                                test_size=0.3, random_state=42)

# Split the remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

def confusion(classifier, X_test, y_test):
    y_pred  = classifier.predict(X_test)
    return confusion_matrix(y_test, y_pred).ravel()

def show(tn,fp,fn,tp):
    print("TN:" + str(tn) + " FP:" + str(fp) + " FN:" + str(fn) + " TP:" + str(tp) + 
          " FNR=" + str(fn/(fn+tp)) + " FPR=" + str(fp/(fp+tn)))
    
w_neg = 10**-4
w_pos_range = np.exp(np.arange(np.log(1), np.log(10**9)))

for w_pos in w_pos_range:
    print("w_pos: " + str(w_pos))
    show(*confusion(RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=10, class_weight={0: w_neg, 1: w_pos}).fit(X_train,y_train),X_test,y_test))


# w_neg = 0.0001
# w_pos = 1096.63    

w_pos: 1.0
TN:3773 FP:64 FN:626 TP:27 FNR=0.9586523736600306 FPR=0.01667969768047954
w_pos: 2.718281828459045
TN:3767 FP:70 FN:623 TP:30 FNR=0.9540581929555896 FPR=0.018243419338024498
w_pos: 7.38905609893065
TN:3744 FP:93 FN:619 TP:34 FNR=0.9479326186830015 FPR=0.024237685691946835
w_pos: 20.085536923187668
TN:3762 FP:75 FN:621 TP:32 FNR=0.9509954058192955 FPR=0.019546520719311962
w_pos: 54.598150033144236
TN:3761 FP:76 FN:626 TP:27 FNR=0.9586523736600306 FPR=0.019807140995569454
w_pos: 148.4131591025766
TN:3715 FP:122 FN:601 TP:52 FNR=0.9203675344563553 FPR=0.03179567370341413
w_pos: 403.4287934927351
TN:3422 FP:415 FN:515 TP:138 FNR=0.7886676875957122 FPR=0.10815741464685953
w_pos: 1096.6331584284585
TN:593 FP:3244 FN:66 TP:587 FNR=0.10107197549770292 FPR=0.8454521761793068
w_pos: 2980.9579870417283
TN:129 FP:3708 FN:6 TP:647 FNR=0.009188361408882083 FPR=0.9663799843627834
w_pos: 8103.083927575384
TN:96 FP:3741 FN:3 TP:650 FNR=0.004594180704441042 FPR=0.9749804534792806
w_pos: 22026

# Comparing different algorithms

In [6]:
# step one: binary label - version(1): without balancing with weights or oversampling
class Binary_classifier:
    def __init__(self, model):
       self.model = model
       

    def readmission_noreadmission_binary_classifier(self):

      #  features = ['Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
      #         'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
      #         'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']
       final_episodes1.columns = [str(col) for col in final_episodes1.columns]

       #features = final_episodes1.iloc[:, 1:-1].columns
       #X = final_episodes1[features]
       
       X = final_episodes1.iloc[:, 1:-1]
       yb = final_episodes1['binary_label']

       # Features in use: 
       print("Features:\n", X.columns)

       # Remove rows with NaN in the target variable
       #X.columns = X.columns.astype(str)
       X = X.copy()
       X.fillna(0, inplace=True)
       X = X[~yb.isna()]
       yb = yb.dropna()

       # Splitting the data into training and testing sets *****
       X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                        yb,
                                                        test_size=0.3, random_state=42)

       # Split the remaining data into validation and test sets
       X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

       # Scaling the features
       scaler = StandardScaler()
       X_train_scaled = scaler.fit_transform(X_train)
       X_val_scaled = scaler.transform(X_val)
       X_test_scaled = scaler.transform(X_test)

       # Store the string identifier of the model
       model_type = self.model  

       #Creating a classifier without making data balanced
       print('Model is',self.model)
       if model_type == 'DecisionTreeClassifier':
             self.model = DecisionTreeClassifier(random_state=42)
       elif model_type == 'LogisticRegression':
             self.model = LogisticRegression(random_state=42, max_iter=1000)
       elif model_type == 'RandomForestClassifier':
             self.model = RandomForestClassifier(random_state=42)
       elif model_type == 'GradientBoostingClassifier':
             self.model = GradientBoostingClassifier(n_estimators=100, random_state=42)
       elif model_type == 'XGBClassifier':
             self.model = XGBClassifier(random_state=42)
       elif model_type == 'MLPClassifier':
             self.model = MLPClassifier(random_state=42, max_iter=1000)

      # Training
       self.model.fit(X_train_scaled, y_train)

       # Making predictions on the test set
       predictions_val = self.model.predict(X_val_scaled)
       predictions_test = self.model.predict(X_test_scaled)

       # Predict for whole data and save the DataFrame as a .csv file
       predicted_BL_df = pd.DataFrame()
       predicted_BL_df["predicted_Binary_label"] = self.model.predict(X)


       predicted_BL_df = pd.concat([df.reset_index(drop=True) for df in [final_episodes1_episode_id, X, predicted_BL_df["predicted_Binary_label"]]], axis=1)
       predicted_BL_df = predicted_BL_df[~pd.isna(predicted_BL_df['predicted_Binary_label'])]
       predicted_BL_df.to_csv("/home/kabank/work/workbench/dipendrp/new-data/Initial_predicted_BL_df.csv") 

        # Evaluate the model
       balanced_accuracy_val = balanced_accuracy_score(y_val, predictions_val)
       f1_val = f1_score(y_val, predictions_val, average='weighted')  
       conf_matrix_val = confusion_matrix(y_val, predictions_val)

       balanced_accuracy_test = balanced_accuracy_score(y_test, predictions_test)
       f1_test = f1_score(y_test, predictions_test, average='weighted') 
       conf_matrix_test = confusion_matrix(y_test, predictions_test)


       print("This is the results:")
       print(f"Balanced Accuracy (Validation): {balanced_accuracy_val}")
       print(f"Balanced Accuracy (Test): {balanced_accuracy_test}")

       print("Confusion Matrix_val:")
       print(conf_matrix_val)
       print("Confusion Matrix_test:")
       Binary_conf_matrix_test = conf_matrix_test
       print(conf_matrix_test)

       # Create a StratifiedKFold cross-validation object *****
       X_train_scaled = pd.DataFrame(X_train_scaled)
       X_val_scaled = pd.DataFrame(X_val_scaled)

       X_train_val_scaled = pd.concat([X_train_scaled, X_val_scaled])
       y_train_val = pd.concat([y_train, y_val])

       cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
       scorer = make_scorer(balanced_accuracy_score)

       # Perform cross-validation on the combined training and validation sets
       cv_scores = cross_val_score(self.model, X_train_val_scaled, y_train_val, cv=cv, scoring=scorer)

       # Display cross-validation results
       print("Cross-Validation Accuracy Scores:")
       print(cv_scores)
       print(f"Mean Accuracy: {np.mean(cv_scores)}")

       print(f"f1_val: {f1_val}")
       print(f"f1_test: {f1_test}")
       # Generate the classification report
       Binary_report = classification_report(y_test, predictions_test)
       print(Binary_report)

       return Binary_report, Binary_conf_matrix_test


In [28]:
# step one: binary label - version(2): with balancing with balanced class_weight 
class Binary_classifier:
    def __init__(self, model):
       self.model = model
       

    def readmission_noreadmission_binary_classifier(self):

      #  features = ['Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
      #         'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
      #         'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']
       final_episodes1.columns = [str(col) for col in final_episodes1.columns]

       #features = final_episodes1.iloc[:, 1:-1].columns
       #X = final_episodes1[features]
       
       X = final_episodes1.iloc[:, 1:-1]
       yb = final_episodes1['binary_label']

       # Features in use: 
       print("Features:\n", X.columns)

       # Remove rows with NaN in the target variable
       #X.columns = X.columns.astype(str)
       X = X.copy()
       X.fillna(0, inplace=True)
       X = X[~yb.isna()]
       yb = yb.dropna()

       # Splitting the data into training and testing sets *****
       X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                        yb,
                                                        test_size=0.3, random_state=42)

       # Split the remaining data into validation and test sets
       X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

       # Scaling the features
       scaler = StandardScaler()
       X_train_scaled = scaler.fit_transform(X_train)
       X_val_scaled = scaler.transform(X_val)
       X_test_scaled = scaler.transform(X_test)


       # Store the string identifier of the model
       model_type = self.model  

      # Creating a classifier with balancing weights and Training the model
       print(model_type)
       if model_type == 'DecisionTreeClassifier':
            self.model = DecisionTreeClassifier(class_weight = 'balanced', random_state=42)
            self.model.fit(X_train_scaled, y_train)
       elif model_type == 'LogisticRegression':
            self.model = LogisticRegression(random_state=42, class_weight= 'balanced', max_iter=1000)
            self.model.fit(X_train_scaled, y_train)
       elif model_type == 'RandomForestClassifier':
            self.model = RandomForestClassifier(class_weight= 'balanced', random_state=42)
            self.model.fit(X_train_scaled, y_train)
       elif model_type == 'GradientBoostingClassifier':
            self.model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
            sample_weight = compute_sample_weight(class_weight='balanced', y=y_train)
            self.model.fit(X_train_scaled, y_train, sample_weight=sample_weight)
       elif model_type == 'XGBClassifier':
            self.model = XGBClassifier(random_state=42)
            sample_weight = compute_sample_weight(class_weight='balanced', y=y_train)
            self.model.fit(X_train_scaled, y_train, sample_weight=sample_weight)
       elif model_type == 'MLPClassifier':
            self.model = MLPClassifier(random_state=42,learning_rate ='adaptive', max_iter=1000)
            self.model.fit(X_train_scaled, y_train) 

       # Making predictions on the test set
       predictions_val = self.model.predict(X_val_scaled)
       predictions_test = self.model.predict(X_test_scaled)

       # Predict for whole data and save the DataFrame as a .csv file
       predicted_BL_df = pd.DataFrame()
       predicted_BL_df["predicted_Binary_label"] = self.model.predict(X)


       predicted_BL_df = pd.concat([df.reset_index(drop=True) for df in [final_episodes1_episode_id, X, predicted_BL_df["predicted_Binary_label"]]], axis=1)
       predicted_BL_df = predicted_BL_df[~pd.isna(predicted_BL_df['predicted_Binary_label'])]
       predicted_BL_df.to_csv("/home/kabank/work/workbench/dipendrp/new-data/Balanced_predicted_BL_df.csv") 

       # Evaluate the model
       balanced_accuracy_val = balanced_accuracy_score(y_val, predictions_val)
       f1_val = f1_score(y_val, predictions_val, average='weighted')  
       conf_matrix_val = confusion_matrix(y_val, predictions_val)

       balanced_accuracy_test = balanced_accuracy_score(y_test, predictions_test)
       f1_test = f1_score(y_test, predictions_test, average='weighted') 
       conf_matrix_test = confusion_matrix(y_test, predictions_test)


       print("This is the results:")
       print(f"Balanced Accuracy (Validation): {balanced_accuracy_val}")
       print(f"Balanced Accuracy (Test): {balanced_accuracy_test}")

       print("Confusion Matrix_val:")
       print(conf_matrix_val)
       print("Confusion Matrix_test:")
       Binary_conf_matrix_test = conf_matrix_test
       print(conf_matrix_test)

       # Create a StratifiedKFold cross-validation object *****
       X_train_scaled = pd.DataFrame(X_train_scaled)
       X_val_scaled = pd.DataFrame(X_val_scaled)

       X_train_val_scaled = pd.concat([X_train_scaled, X_val_scaled])
       y_train_val = pd.concat([y_train, y_val])

       cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
       scorer = make_scorer(balanced_accuracy_score)

       # Perform cross-validation on the combined training and validation sets
       cv_scores = cross_val_score(self.model, X_train_val_scaled, y_train_val, cv=cv, scoring=scorer)

       # Display cross-validation results
       print("Cross-Validation Accuracy Scores:")
       print(cv_scores)
       print(f"Mean Accuracy: {np.mean(cv_scores)}")

       print(f"f1_val: {f1_val}")
       print(f"f1_test: {f1_test}")
       # Generate the classification report
       Binary_report = classification_report(y_test, predictions_test)
       print(Binary_report)

       return Binary_report, Binary_conf_matrix_test
        

In [24]:
# step one: binary label - version(3): with balancing with Custom weighting
class Binary_classifier:
    def __init__(self, model):
       self.model = model
       

    def readmission_noreadmission_binary_classifier(self):

      #  features = ['Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
      #         'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
      #         'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']
       final_episodes1.columns = [str(col) for col in final_episodes1.columns]

       #features = final_episodes1.iloc[:, 1:-1].columns
       #X = final_episodes1[features]
       
       X = final_episodes1.iloc[:, 1:-1]
       yb = final_episodes1['binary_label']

       # Features in use: 
       print("Features:\n", X.columns)

       # Remove rows with NaN in the target variable
       #X.columns = X.columns.astype(str)
       X = X.copy()
       X.fillna(0, inplace=True)
       X = X[~yb.isna()]
       yb = yb.dropna()

       # Splitting the data into training and testing sets *****
       X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                        yb,
                                                        test_size=0.3, random_state=42)

       # Split the remaining data into validation and test sets
       X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

       # Scaling the features
       scaler = StandardScaler()
       X_train_scaled = scaler.fit_transform(X_train)
       X_val_scaled = scaler.transform(X_val)
       X_test_scaled = scaler.transform(X_test)

       # Calculate the class weight 
       # for LogisticRegression
       wl1 = (0.95)
       wl0 = wl1/(counts[0]/counts[1])
       # for decision tree
       wd1 = (max_val_bin / counts[1]).round(2)
       wd0 = (max_val_bin / counts[0]).round(2)

       # for Random Forest (we chose these weights from above)
       w_neg = 0.0001
       w_pos = 1096.63 

       # GradientBoosting
       GB1 = (counts[0]/counts[1])
       GB0 = (counts[0]/counts[0])
      # Create a sample_weight array that is all ones
       sample_weight = np.ones(y_train.shape)
      # Set the weights for class 0
       sample_weight[y_train == 0] = GB0
      # Set the weights for class 1
       sample_weight[y_train == 1] = GB1

       # for XGBClassifier
       ratio = float(counts[0] / counts[1])

      # learning_rate:[0.01,0.05,0.1]
      # n_estimators: np.arange(100,500,100)

       # Store the string identifier of the model
       model_type = self.model  

      # Creating a classifier with balancing weights and Training the model
       print(model_type)
       if model_type == 'DecisionTreeClassifier':
            self.model = DecisionTreeClassifier(class_weight = {0:wd0, 1:wd1}, random_state=42)
            self.model.fit(X_train_scaled, y_train)
       elif model_type == 'LogisticRegression':
            self.model = LogisticRegression(solver='newton-cg', class_weight={0: wl0, 1: wl1}, max_iter=1000,random_state=42)
            self.model.fit(X_train_scaled, y_train)
       elif model_type == 'RandomForestClassifier':
            self.model = RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=10, class_weight={0: w_neg, 1: w_pos})
            self.model.fit(X_train_scaled, y_train)
       elif model_type == 'GradientBoostingClassifier':
            self.model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
            self.model.fit(X_train_scaled, y_train, sample_weight=sample_weight)
       elif model_type == 'XGBClassifier':
            self.model = XGBClassifier(scale_pos_weight=ratio,random_state=42)
            self.model.fit(X_train_scaled, y_train) 
          #   sample_weight = compute_sample_weight(class_weight='balanced', y=y_train) # not making so much difference
          #   self.model.fit(X_train_scaled, y_train, sample_weight=sample_weight)
       elif model_type == 'MLPClassifier':
            self.model = MLPClassifier(learning_rate ='adaptive', max_iter=1000, random_state=42)
            self.model.fit(X_train_scaled, y_train) 

       # Making predictions on the test set
       predictions_val = self.model.predict(X_val_scaled)
       predictions_test = self.model.predict(X_test_scaled)

       # Predict for whole data and save the DataFrame as a .csv file
       predicted_BL_df = pd.DataFrame()
       predicted_BL_df["predicted_Binary_label"] = self.model.predict(X)


       predicted_BL_df = pd.concat([df.reset_index(drop=True) for df in [final_episodes1_episode_id, X, predicted_BL_df["predicted_Binary_label"]]], axis=1)
       predicted_BL_df = predicted_BL_df[~pd.isna(predicted_BL_df['predicted_Binary_label'])]
       predicted_BL_df.to_csv("/home/kabank/work/workbench/dipendrp/new-data/Custom_weight_predicted_BL_df.csv") 

       # Evaluate the model
       balanced_accuracy_val = balanced_accuracy_score(y_val, predictions_val)
       f1_val = f1_score(y_val, predictions_val, average='weighted')  
       conf_matrix_val = confusion_matrix(y_val, predictions_val)

       balanced_accuracy_test = balanced_accuracy_score(y_test, predictions_test)
       f1_test = f1_score(y_test, predictions_test, average='weighted') 
       conf_matrix_test = confusion_matrix(y_test, predictions_test)


       print("This is the results:")
       print(f"Balanced Accuracy (Validation): {balanced_accuracy_val}")
       print(f"Balanced Accuracy (Test): {balanced_accuracy_test}")

       print("Confusion Matrix_val:")
       print(conf_matrix_val)
       print("Confusion Matrix_test:")
       Binary_conf_matrix_test = conf_matrix_test
       print(conf_matrix_test)

       # Create a StratifiedKFold cross-validation object *****
       X_train_scaled = pd.DataFrame(X_train_scaled)
       X_val_scaled = pd.DataFrame(X_val_scaled)

       X_train_val_scaled = pd.concat([X_train_scaled, X_val_scaled])
       y_train_val = pd.concat([y_train, y_val])

       cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
       scorer = make_scorer(balanced_accuracy_score)

       # Perform cross-validation on the combined training and validation sets
       cv_scores = cross_val_score(self.model, X_train_val_scaled, y_train_val, cv=cv, scoring=scorer)

       # Display cross-validation results
       print("Cross-Validation Accuracy Scores:")
       print(cv_scores)
       print(f"Mean Accuracy: {np.mean(cv_scores)}")

       print(f"f1_val: {f1_val}")
       print(f"f1_test: {f1_test}")
       # Generate the classification report
       Binary_report = classification_report(y_test, predictions_test)
       print(Binary_report)

       return Binary_report, Binary_conf_matrix_test
        

In [10]:
# step one: binary label - version(4): balancing with oversampling
class Binary_classifier:
    def __init__(self, model):
       self.model = model
       

    def readmission_noreadmission_binary_classifier(self):

      #  features = ['Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
      #         'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
      #         'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']
       final_episodes1.columns = [str(col) for col in final_episodes1.columns]

       #features = final_episodes1.iloc[:, 1:-1].columns
       #X = final_episodes1[features]
       
       X = final_episodes1.iloc[:, 1:-1]
       yb = final_episodes1['binary_label']

       # Features in use: 
       print("Features:\n", X.columns)

       # Remove rows with NaN in the target variable
       #X.columns = X.columns.astype(str)
       X = X.copy()
       X.fillna(0, inplace=True)
       X = X[~yb.isna()]
       yb = yb.dropna()

       # Splitting the data into training and testing sets *****
       X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                        yb,
                                                        test_size=0.3, random_state=42)

       # Split the remaining data into validation and test sets
       X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

       # Scaling the features
       scaler = StandardScaler()
       X_train_scaled = scaler.fit_transform(X_train)
       X_val_scaled = scaler.transform(X_val)
       X_test_scaled = scaler.transform(X_test)

       #Creating a classifier without making data balanced
       print(self.model)
       if self.model == 'DecisionTreeClassifier':
             self.model = DecisionTreeClassifier(random_state=42)
       elif self.model == 'LogisticRegression':
             self.model = LogisticRegression(random_state=42, max_iter=1000)
       elif self.model == 'RandomForestClassifier':
             self.model = RandomForestClassifier(random_state=42)
       elif self.model == 'GradientBoostingClassifier':
             self.model = GradientBoostingClassifier(n_estimators=100, random_state=42)
       elif self.model == 'XGBClassifier':
             self.model = XGBClassifier(random_state=42)
       elif self.model == 'MLPClassifier':
             self.model = MLPClassifier(random_state=42, max_iter=1000)

       # Oversampling and fitting
       ros = RandomOverSampler(random_state=0)
       X_resampled, y_resampled = ros.fit_resample(X, yb)   
       X_train_scaled = X_resampled
       y_train = y_resampled

       self.model.fit(X_train_scaled, y_train)

       # Making predictions on the test set
       predictions_val = self.model.predict(X_val_scaled)
       predictions_test = self.model.predict(X_test_scaled)

       # Predict for whole data and save the DataFrame as a .csv file
       predicted_BL_df = pd.DataFrame()
       predicted_BL_df["predicted_Binary_label"] = self.model.predict(X)


       predicted_BL_df = pd.concat([df.reset_index(drop=True) for df in [final_episodes1_episode_id, X, predicted_BL_df["predicted_Binary_label"]]], axis=1)
       predicted_BL_df = predicted_BL_df[~pd.isna(predicted_BL_df['predicted_Binary_label'])]
       predicted_BL_df.to_csv("/home/kabank/work/workbench/dipendrp/new-data/Oversampling_predicted_BL_df.csv") 

       # Evaluate the model
       balanced_accuracy_val = balanced_accuracy_score(y_val, predictions_val)
       f1_val = f1_score(y_val, predictions_val, average='weighted')  
       conf_matrix_val = confusion_matrix(y_val, predictions_val)

       balanced_accuracy_test = balanced_accuracy_score(y_test, predictions_test)
       f1_test = f1_score(y_test, predictions_test, average='weighted') 
       conf_matrix_test = confusion_matrix(y_test, predictions_test)


       print("This is the results:")
       print(f"Balanced Accuracy (Validation): {balanced_accuracy_val}")
       print(f"Balanced Accuracy (Test): {balanced_accuracy_test}")

       print("Confusion Matrix_val:")
       print(conf_matrix_val)
       print("Confusion Matrix_test:")
       Binary_conf_matrix_test = conf_matrix_test
       print(conf_matrix_test)

      #  # Create a StratifiedKFold cross-validation object *****
      #  X_train_scaled = pd.DataFrame(X_train_scaled)
      #  X_val_scaled = pd.DataFrame(X_val_scaled)

      #  X_train_val_scaled = pd.concat([X_train_scaled, X_val_scaled])
      #  y_train_val = pd.concat([y_train, y_val])

      #  cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      #  scorer = make_scorer(balanced_accuracy_score)

      #  # Perform cross-validation on the combined training and validation sets
      #  cv_scores = cross_val_score(self.model, X_train_val_scaled, y_train_val, cv=cv, scoring=scorer)

      #  # Display cross-validation results
      #  print("Cross-Validation Accuracy Scores:")
      #  print(cv_scores)
      #  print(f"Mean Accuracy: {np.mean(cv_scores)}")

       print(f"f1_val: {f1_val}")
       print(f"f1_test: {f1_test}")
       # Generate the classification report
       Binary_report = classification_report(y_test, predictions_test)
       print(Binary_report)

       return Binary_report, Binary_conf_matrix_test 


In [29]:
# step two - multi_label version(1): without balancing with weights or oversampling
class Multi_classifier:
    def __init__(self, model):
       self.model = model
       

    def readmission_multi_classifier(self):
      #  features = ['Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
      #         'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
      #         'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']
       features = final_episodes2.iloc[:, 1:-2].columns

       X = final_episodes2[features]
       
       ym= final_episodes2['multi_label']

       # Features in use: 
       print("Features:\n", X.columns)
       print(X.shape)

       # Remove rows with NaN in the target variable
       X = X.copy()
       X.fillna(0, inplace=True)
       X = X[~ym.isna()]
       ym = ym.dropna()

       # Splitting the data into training and testing sets *****
       X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                        ym,
                                                        test_size=0.3, random_state=42)

       # Split the remaining data into validation and test sets
       X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

       # Scaling the features
       scaler = StandardScaler()
       X_train_scaled = scaler.fit_transform(X_train)
       X_val_scaled = scaler.transform(X_val)
       X_test_scaled = scaler.transform(X_test)

       #Creating a classifier without making data balanced
       print(self.model)
       if self.model == 'DecisionTreeClassifier':
             self.model = DecisionTreeClassifier(random_state=42)
       elif self.model == 'LogisticRegression':
             self.model = LogisticRegression(random_state=42, max_iter=1000)
       elif self.model == 'RandomForestClassifier':
             self.model = RandomForestClassifier(random_state=42)
       elif self.model == 'GradientBoostingClassifier':
             self.model = GradientBoostingClassifier(n_estimators=100, random_state=42)
       elif self.model == 'XGBClassifier':
             self.model = XGBClassifier(random_state=42)
       elif self.model == 'MLPClassifier':
             self.model = MLPClassifier(random_state=42, max_iter=1000)

      # Training
       self.model.fit(X_train_scaled, y_train)

       # Making predictions on the test set
       predictions_val = self.model.predict(X_val_scaled)
       predictions_test = self.model.predict(X_test_scaled)

       # Predict for whole data and save the DataFrame as a .csv file
       predicted_ML_df = pd.DataFrame()
       predicted_ML_df["predicted_Multi_label"] = self.model.predict(X)

       #X2_with_episodeid = pd.concat([final_episodes2_episode_id,X], axis=1)
       predicted_ML_df = pd.concat([df.reset_index(drop=True) for df in [final_episodes2_episode_id, X, predicted_ML_df["predicted_Multi_label"]]], axis=1)
       # saving the labeled results for later
       predicted_ML_df.to_csv("/home/kabank/work/workbench/dipendrp/new-data/Initial_predicted_ML_df.csv") 

       # Evaluate the model
       balanced_accuracy_val = balanced_accuracy_score(y_val, predictions_val)
       f1_val = f1_score(y_val, predictions_val, average='weighted')  
       conf_matrix_val = confusion_matrix(y_val, predictions_val)

       balanced_accuracy_test = balanced_accuracy_score(y_test, predictions_test)
       f1_test = f1_score(y_test, predictions_test, average='weighted') 
       conf_matrix_test = confusion_matrix(y_test, predictions_test)


       print("This is the results:")
       print(f"Balanced Accuracy (Validation): {balanced_accuracy_val}")
       print(f"Balanced Accuracy (Test): {balanced_accuracy_test}")

       print("Confusion Matrix_val:")
       print(conf_matrix_val)
       print("Confusion Matrix_test:")
       Multi_conf_matrix_test = conf_matrix_test
       print(conf_matrix_test)

       # Create a StratifiedKFold cross-validation object *****
       X_train_scaled = pd.DataFrame(X_train_scaled)
       X_val_scaled = pd.DataFrame(X_val_scaled)

       X_train_val_scaled = pd.concat([X_train_scaled, X_val_scaled])
       y_train_val = pd.concat([y_train, y_val])

       cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
       scorer = make_scorer(balanced_accuracy_score)

       # Perform cross-validation on the combined training and validation sets
       cv_scores = cross_val_score(self.model, X_train_val_scaled, y_train_val, cv=cv, scoring=scorer)

       # Display cross-validation results
       print("Cross-Validation Accuracy Scores:")
       print(cv_scores)
       print(f"Mean Accuracy: {np.mean(cv_scores)}")

       print(f"f1_val: {f1_val}")
       print(f"f1_test: {f1_test}")
       # Generate the classification report
       Multi_report = classification_report(y_test, predictions_test)
       print(Multi_report)

       return Multi_conf_matrix_test, Multi_report


In [76]:
# step two - multi_label version(2): with balancing with balanced class_weightx
class Multi_classifier:
    def __init__(self, model):
       self.model = model
       

    def readmission_multi_classifier(self):
      #  features = ['Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
      #         'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
      #         'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']
       features = final_episodes2.iloc[:, 1:-2].columns

       X = final_episodes2[features]
       
       ym= final_episodes2['multi_label']

       # Features in use: 
       print("Features:\n", X.columns)
       print(X.shape)

       # Remove rows with NaN in the target variable
       X = X.copy()
       X.fillna(0, inplace=True)
       X = X[~ym.isna()]
       ym = ym.dropna()

       # Splitting the data into training and testing sets *****
       X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                        ym,
                                                        test_size=0.3, random_state=42)

       # Split the remaining data into validation and test sets
       X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

       # Scaling the features
       scaler = StandardScaler()
       X_train_scaled = scaler.fit_transform(X_train)
       X_val_scaled = scaler.transform(X_val)
       X_test_scaled = scaler.transform(X_test)
       
       # Calculate the ratio 
       ratio = [(max_val_multi / counts[0]).round(2), (max_val_multi / counts[1]).round(2), (max_val_multi / counts[2]).round(2)]

       # Store the string identifier of the model
       model_type = self.model  

      # Creating a classifier with balancing weights and Training the model
       print(model_type)
       if model_type == 'DecisionTreeClassifier':
            self.model = DecisionTreeClassifier(class_weight = 'balanced', random_state=42)
            self.model.fit(X_train_scaled, y_train)
       elif model_type == 'LogisticRegression':
            self.model = LogisticRegression(random_state=42, class_weight= 'balanced', max_iter=1000)
            self.model.fit(X_train_scaled, y_train)
       elif model_type == 'RandomForestClassifier':
            self.model = RandomForestClassifier(class_weight= 'balanced', random_state=42)
            self.model.fit(X_train_scaled, y_train)
       elif model_type == 'GradientBoostingClassifier':
            self.model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
            sample_weight = compute_sample_weight(class_weight='balanced', y=y_train)
            self.model.fit(X_train_scaled, y_train, sample_weight=sample_weight)
       elif model_type == 'XGBClassifier':
            self.model = XGBClassifier(random_state=42)
            sample_weight = compute_sample_weight(class_weight='balanced', y=y_train)
            self.model.fit(X_train_scaled, y_train, sample_weight=sample_weight)
       elif model_type == 'MLPClassifier':
            self.model = MLPClassifier(random_state=42,learning_rate ='adaptive', max_iter=1000)
            self.model.fit(X_train_scaled, y_train) 

       # Making predictions on the test set
       predictions_val = self.model.predict(X_val_scaled)
       predictions_test = self.model.predict(X_test_scaled)

       # Predict for whole data and save the DataFrame as a .csv file
       predicted_ML_df = pd.DataFrame()
       predicted_ML_df["predicted_Multi_label"] = self.model.predict(X)

       #X2_with_episodeid = pd.concat([final_episodes2_episode_id,X], axis=1)
       predicted_ML_df = pd.concat([df.reset_index(drop=True) for df in [final_episodes2_episode_id, X, predicted_ML_df["predicted_Multi_label"]]], axis=1)
       predicted_ML_df.to_csv("/home/kabank/work/workbench/dipendrp/new-data/Balanced_predicted_ML_df.csv") 

       # Evaluate the model
       balanced_accuracy_val = balanced_accuracy_score(y_val, predictions_val)
       f1_val = f1_score(y_val, predictions_val, average='weighted')  
       conf_matrix_val = confusion_matrix(y_val, predictions_val)

       balanced_accuracy_test = balanced_accuracy_score(y_test, predictions_test)
       f1_test = f1_score(y_test, predictions_test, average='weighted') 
       conf_matrix_test = confusion_matrix(y_test, predictions_test)


       print("This is the results:")
       print(f"Balanced Accuracy (Validation): {balanced_accuracy_val}")
       print(f"Balanced Accuracy (Test): {balanced_accuracy_test}")

       print("Confusion Matrix_val:")
       print(conf_matrix_val)
       print("Confusion Matrix_test:")
       Multi_conf_matrix_test = conf_matrix_test
       print(conf_matrix_test)

       # Create a StratifiedKFold cross-validation object *****
       X_train_scaled = pd.DataFrame(X_train_scaled)
       X_val_scaled = pd.DataFrame(X_val_scaled)

       X_train_val_scaled = pd.concat([X_train_scaled, X_val_scaled])
       y_train_val = pd.concat([y_train, y_val])

       cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
       scorer = make_scorer(balanced_accuracy_score)

       # Perform cross-validation on the combined training and validation sets
       cv_scores = cross_val_score(self.model, X_train_val_scaled, y_train_val, cv=cv, scoring=scorer)

       # Display cross-validation results
       print("Cross-Validation Accuracy Scores:")
       print(cv_scores)
       print(f"Mean Accuracy: {np.mean(cv_scores)}")

       print(f"f1_val: {f1_val}")
       print(f"f1_test: {f1_test}")
       # Generate the classification report
       Multi_report = classification_report(y_test, predictions_test)
       print(Multi_report)

       return Multi_conf_matrix_test, Multi_report


In [36]:
# step two - multi_label version(3): with balancing with oversampling
class Multi_classifier:
    def __init__(self, model):
       self.model = model
       

    def readmission_multi_classifier(self):
      #  features = ['Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
      #         'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
      #         'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']
       features = final_episodes2.iloc[:, 1:-2].columns

       X = final_episodes2[features]
       
       ym= final_episodes2['multi_label']

       # Features in use: 
       print("Features:\n", X.columns)
       print(X.shape)

       # Remove rows with NaN in the target variable
       X = X.copy()
       X.fillna(0, inplace=True)
       X = X[~ym.isna()]
       ym = ym.dropna()

       # Splitting the data into training and testing sets *****
       X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                        ym,
                                                        test_size=0.3, random_state=42)

       # Split the remaining data into validation and test sets
       X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=42)

       # Scaling the features
       scaler = StandardScaler()
       X_train_scaled = scaler.fit_transform(X_train)
       X_val_scaled = scaler.transform(X_val)
       X_test_scaled = scaler.transform(X_test)

       #Creating a classifier without making data balanced
       print(self.model)
       if self.model == 'DecisionTreeClassifier':
             self.model = DecisionTreeClassifier(random_state=42)
       elif self.model == 'LogisticRegression':
             self.model = LogisticRegression(random_state=42, max_iter=1000)
       elif self.model == 'RandomForestClassifier':
             self.model = RandomForestClassifier(random_state=42)
       elif self.model == 'GradientBoostingClassifier':
             self.model = GradientBoostingClassifier(n_estimators=100, random_state=42)
       elif self.model == 'XGBClassifier':
             self.model = XGBClassifier(random_state=42)
       elif self.model == 'MLPClassifier':
             self.model = MLPClassifier(random_state=42, max_iter=1000)

       # Oversampling and fitting
       ros = RandomOverSampler(random_state=0)
       X_resampled, y_resampled = ros.fit_resample(X, ym)   
       X_train_scaled = X_resampled
       y_train = y_resampled

       self.model.fit(X_train_scaled, y_train)

       # Making predictions on the test set
       predictions_val = self.model.predict(X_val_scaled)
       predictions_test = self.model.predict(X_test_scaled)

       # Predict for whole data and save the DataFrame as a .csv file
       predicted_ML_df = pd.DataFrame()
       predicted_ML_df["predicted_Multi_label"] = self.model.predict(X)

       #X2_with_episodeid = pd.concat([final_episodes2_episode_id,X], axis=1)
       predicted_ML_df = pd.concat([df.reset_index(drop=True) for df in [final_episodes2_episode_id, X, predicted_ML_df["predicted_Multi_label"]]], axis=1)
       predicted_ML_df.to_csv("/home/kabank/work/workbench/dipendrp/new-data/Oversampling_predicted_ML_df.csv") 

       # Evaluate the model
       balanced_accuracy_val = balanced_accuracy_score(y_val, predictions_val)
       f1_val = f1_score(y_val, predictions_val, average='weighted')  
       conf_matrix_val = confusion_matrix(y_val, predictions_val)

       balanced_accuracy_test = balanced_accuracy_score(y_test, predictions_test)
       f1_test = f1_score(y_test, predictions_test, average='weighted') 
       conf_matrix_test = confusion_matrix(y_test, predictions_test)


       print("This is the results:")
       print(f"Balanced Accuracy (Validation): {balanced_accuracy_val}")
       print(f"Balanced Accuracy (Test): {balanced_accuracy_test}")

       print("Confusion Matrix_val:")
       print(conf_matrix_val)
       print("Confusion Matrix_test:")
       Multi_conf_matrix_test = conf_matrix_test
       print(conf_matrix_test)

      #  # Create a StratifiedKFold cross-validation object *****
      #  X_train_scaled = pd.DataFrame(X_train_scaled)
      #  X_val_scaled = pd.DataFrame(X_val_scaled)

      #  X_train_val_scaled = pd.concat([X_train_scaled, X_val_scaled])
      #  y_train_val = pd.concat([y_train, y_val])

      #  cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      #  scorer = make_scorer(balanced_accuracy_score)

      #  # Perform cross-validation on the combined training and validation sets
      #  cv_scores = cross_val_score(self.model, X_train_val_scaled, y_train_val, cv=cv, scoring=scorer)

      #  # Display cross-validation results
      #  print("Cross-Validation Accuracy Scores:")
      #  print(cv_scores)
      #  print(f"Mean Accuracy: {np.mean(cv_scores)}")

       print(f"f1_val: {f1_val}")
       print(f"f1_test: {f1_test}")
       # Generate the classification report
       Multi_report = classification_report(y_test, predictions_test)
       print(Multi_report)

       return Multi_conf_matrix_test, Multi_report


In [37]:
#options for algorithms: DecisionTreeClassifier, LogisticRegression, RandomForestClassifier, GradientBoostingClassifier, XGBClassifier, MLPClassifier

binary_classifier_ins = Binary_classifier('XGBClassifier').readmission_noreadmission_binary_classifier()
print("\n")
multi_classifier_ins = Multi_classifier('LogisticRegression').readmission_multi_classifier()

#Final choices:  
#Binary_classifier --> XGBClassifier with class weight
#Multi_classifier --> Logistic Regression with oversampling




Features:
 Index(['Length_of_Episode', 'Count_visit', 'SD_CareEvent_PerMonth',
       'num_diagnoses', 'num_medications', 'Inpatient_day_ratio',
       'Outpatient_ratio', 'Therapy_ratio', 'TreatmentPlanning_ratio',
       'Advisory_ratio',
       ...
       'G03DA02', 'B03BA03', 'G03FB05', 'S01AA13', 'A11JB', 'M01AE01',
       'N05AE04', 'N06AA09', 'R03AC03', 'B03BA01'],
      dtype='object', length=216)
(3426, 216)
LogisticRegression
This is the results:
Balanced Accuracy (Validation): 0.507104224297291
Balanced Accuracy (Test): 0.48369590826985
Confusion Matrix_val:
[[51 18 29]
 [46 34 62]
 [13 13 83]]
Confusion Matrix_test:
[[106  30  74]
 [ 79  59 117]
 [ 34  27 153]]
f1_val: 0.454559974391343
f1_test: 0.445119563740356
              precision    recall  f1-score   support

           0       0.48      0.50      0.49       210
           1       0.51      0.23      0.32       255
           2       0.44      0.71      0.55       214

    accuracy                           0.47  

/home/kabank/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kabank/anaconda3/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/kabank/anaconda3/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [38]:
# check the created CSVs
# df_I_BL = pd.read_csv('/home/kabank/work/workbench/dipendrp/new-data/Initial_predicted_BL_df.csv')
df_B_BL = pd.read_csv('/home/kabank/work/workbench/dipendrp/new-data/Balanced_predicted_BL_df.csv') # the binary model which became balanced by XGBClassifier with class weight
# df_CW_BL = pd.read_csv('/home/kabank/work/workbench/dipendrp/new-data/Custom_weight_predicted_BL_df.csv')
# df_OS_BL = pd.read_csv('/home/kabank/work/workbench/dipendrp/new-data/Oversampling_predicted_BL_df.csv')
print(len(df_B_BL))
print(df_B_BL['predicted_Binary_label'].value_counts())
# print(df_BL.head())

# df_I_ML = pd.read_csv('/home/kabank/work/workbench/dipendrp/new-data/Initial_predicted_ML_df.csv')
# df_B_ML = pd.read_csv('/home/kabank/work/workbench/dipendrp/new-data/Balanced_predicted_ML_df.csv')
# df_CW_ML = pd.read_csv('/home/kabank/work/workbench/dipendrp/new-data/Custom_weight_predicted_ML_df.csv')
df_OS_ML = pd.read_csv('/home/kabank/work/workbench/dipendrp/new-data/Oversampling_predicted_ML_df.csv') # the multi-class model which became oversampled by Logistic Regression with oversampling
print(len(df_OS_ML))
# print(df_ML.columns)
print(df_OS_ML['predicted_Multi_label'].value_counts())
# print(df_ML['episode_id'].nunique())


22676
predicted_Binary_label
0    22450
1      226
Name: count, dtype: int64
3426
predicted_Multi_label
2    1639
0    1076
1     711
Name: count, dtype: int64


In [179]:
# check the length of created dataframes
print(len(final_episodes))
print(len(final_episodes1))
print(len(final_episodes2))
print(len(final_episodes1_episode_id))
print(len(final_episodes2_episode_id))

22676
22676
3426
22676
3426


# Comparing with initial solution
in the initial solution, we gave these six labels to the data: <br>
labels' value count: label <br>
5    19250 <br>
0      878 <br>
1      721 <br>
2      718 <br>
4      662 <br>
3      447

In [5]:
# read data *****
final_episodes = pd.read_csv("/home/kabank/workbench/.conda/analysis/kabank-data/final_episodes.csv")
fullHot_episodes = pd.read_csv("/home/kabank/workbench/.conda/analysis/kabank-data/final_episodes4.csv") 
Diag_med = pd.read_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/Dummies_ICD10_ATC_100.csv') #Using the columns for diagnoses (in ICD-10) and medications (in ATC level 5)

# required changes *****
final_episodes.rename(columns={'var_no_dates_permonth': 'SD_CareEvent_PerMonth'}, inplace=True)


#merge to add some required columns *****
final_episodes = pd.merge(final_episodes, fullHot_episodes[['episode_id', 'num_diagnoses', 'num_medications', 'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']], on='episode_id', how='inner')
final_episodes = pd.merge(final_episodes[['episode_id','tillnextepisode','Length_of_Episode','Count_visit','SD_CareEvent_PerMonth', 'num_diagnoses', 'num_medications',
       'Inpatient_day_ratio', 'Outpatient_ratio', 'Therapy_ratio','TreatmentPlanning_ratio', 'Advisory_ratio',
       'gender_0','F','M','MiddleChildhood','Preschooler','Teenager']], Diag_med, on='episode_id', how='left')

# Encoding 'tillnextepisode' into labels:  
le = LabelEncoder()
final_episodes['label'] = le.fit_transform(pd.cut(final_episodes['tillnextepisode'],
                                                  bins=[float('-inf'), 0, 180, 365, 730, 1095, float('inf')],
                                                  labels=['not-re-admitted',
                                                          're-admitted in 0-180 days',
                                                          're-admitted in 180-365 days',
                                                          're-admitted in 365-730 days',
                                                          're-admitted in 730-1095 days',
                                                          're-admitted in more than 1095 days']))
# labels with value counts *****
print("labels' value count:", final_episodes['label'].value_counts())
print(final_episodes['label'].unique())

# Selecting independent features
features = final_episodes.iloc[:, 2:-1].columns

X = final_episodes[features]
y= final_episodes['label']

# Features in use: 
print("Features:\n", X.columns)

# Remove rows with NaN in the target variable
X = X.copy()
X.fillna(0, inplace=True)
X = X[~y.isna()]
y = y.dropna()

# Splitting the data into training and testing sets *****
X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                    y,
                                                    test_size=0.2, random_state=42)

# Split the remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y),y=y)
class_weight = dict(zip(np.unique(final_episodes['label']), class_weights))

# Creating a decision tree classifier with class weights
NN = MLPClassifier(random_state=42,learning_rate ='adaptive', max_iter=1000)

# Training the model
NN.fit(X_train_scaled, y_train)

# Making predictions on the test set
predictions_val = NN.predict(X_val_scaled)
predictions_test = NN.predict(X_test_scaled)

# Evaluate the model
balanced_accuracy_val = balanced_accuracy_score(y_val, predictions_val)
f1_val = f1_score(y_val, predictions_val, average='weighted')  # Using 'weighted' for multiclass problems
conf_matrix_val = confusion_matrix(y_val, predictions_val)

balanced_accuracy_test = balanced_accuracy_score(y_test, predictions_test)
f1_test = f1_score(y_test, predictions_test, average='weighted')  # Using 'weighted' for multiclass problems
conf_matrix_test = confusion_matrix(y_test, predictions_test)


print("This is the results for With diagnoses and medications and with DecisionTreeClassifier:")
print(f"Balanced Accuracy (Validation): {balanced_accuracy_val}")
print(f"Balanced Accuracy (Test): {balanced_accuracy_test}")

print("Confusion Matrix_val:")
print(conf_matrix_val)
print("Confusion Matrix_test:")
print(conf_matrix_test)

# Create a StratifiedKFold cross-validation object *****
X_train_scaled = pd.DataFrame(X_train_scaled)
X_val_scaled = pd.DataFrame(X_val_scaled)

X_train_val_scaled = pd.concat([X_train_scaled, X_val_scaled])
y_train_val = pd.concat([y_train, y_val])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(balanced_accuracy_score)

# Perform cross-validation on the combined training and validation sets
cv_scores = cross_val_score(NN, X_train_val_scaled, y_train_val, cv=cv, scoring=scorer)

# Display cross-validation results
print("Cross-Validation Accuracy Scores:")
print(cv_scores)
print(f"Mean Accuracy: {np.mean(cv_scores)}")

print(f"f1_val: {f1_val}")
print(f"f1_test: {f1_test}")
# Generate the classification report
report = classification_report(y_test, predictions_test)

print(report)

labels' value count: label
5    19250
0      878
1      721
2      718
4      662
3      447
Name: count, dtype: int64
[5 1 4 2 3 0]
Features:
 Index(['Length_of_Episode', 'Count_visit', 'SD_CareEvent_PerMonth',
       'num_diagnoses', 'num_medications', 'Inpatient_day_ratio',
       'Outpatient_ratio', 'Therapy_ratio', 'TreatmentPlanning_ratio',
       'Advisory_ratio',
       ...
       'G03DA02', 'B03BA03', 'G03FB05', 'S01AA13', 'A11JB', 'M01AE01',
       'N05AE04', 'N06AA09', 'R03AC03', 'B03BA01'],
      dtype='object', length=216)
This is the results for With diagnoses and medications and with DecisionTreeClassifier:
Balanced Accuracy (Validation): 0.1777054776183337
Balanced Accuracy (Test): 0.18324424962216312
Confusion Matrix_val:
[[   1    2    1    0    0   88]
 [   1    1    3    0    2   80]
 [   1    1    0    1    3   50]
 [   1    2    1    2    1   43]
 [   1    0    0    1    3   58]
 [  25   15   20   11   13 1836]]
Confusion Matrix_test:
[[   3    1    4    2    0   